Notebook walking through constructing a Dataset with DatasetBuilder

In [4]:
import sys
import pandas as pd
sys.path.append("..")

from chemspace.Dataset.DatasetBuilder import DatasetBuilder

Dataset Builder Class can be instantiated one of 3 ways:
1. From a Chemical Structures Records `.json.gz` file downloaded from PubChem's search function
2. From a `.csv` file containing CIDs as the indices of the file
3. From a previously constructed DataFrame that has CIDs of interest as the indices

Method 1: Instantiate a DatasetBuilder object from a Chemical Structures Records `.json.gz` file downloaded from PubChem's search function

In [8]:
# Instantiate class with json file form PubChem
DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/PubChem_compound_list_records.json.gz')

# Display dataset
DB.dataset

# Save as .CSV
#DB.dataset.to_csv('../chemspace/Dataset/Data/CIDs.csv', index = True)

Method 2: Instantiate a DatasetBuilder object from a `.csv` file containing CIDs as the indices of the file

In [6]:
# Instantiate class with previously generated Dataframe (CSV)
DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/CIDs.csv')

# Display dataset
DB.dataset

""
1
4
6
9
12
...
168010239
168010263
168010365
168010385


Method 3: Instantiate a DatasetBuilder object from a previously constructed DataFrame that has CIDs of interest as the indices

In [7]:
# Load df
df = pd.read_csv('../chemspace/Dataset/Data/CIDs.csv', index_col='Unnamed: 0')

# Instantiate class with previously generated Dataframe
DB = DatasetBuilder(compound_df=df)

# Display dataset
DB.dataset

""
1
4
6
9
12
...
168010239
168010263
168010365
168010385
